# CMR User Configurator

To use this system with its default settings, you will need an account from TACC. If you don't already have a TACC login, please go there now and sign up.

The simplest way to use the User Configurator for the Coastal Model Repository (CMR), is to execute the cells below, one at a time.

In [1]:
from setvar import *
import os, re, sys

In [2]:
# Some settings needed by the system
setvar("""
AGAVE_JSON_PARSER=jq
PATH=$HOME/agave-model/cli/bin:$PATH
""")

AGAVE_JSON_PARSER=jq
PATH=/home/jupuser/agave-model/cli/bin:/home/jupuser/agave-model/cli/bin:/home/jupuser/agave-model/cli/bin:/home/jupuser/agave-model/cli/bin:/home/jupuser/agave-model/cli/bin:/home/jupuser/agave-model/bin:/home/jupuser/agave-model/cli/bin:/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin


## Step 1: Choose your tenant
The tenant is the system that manages your login credentials and Agave data. You
can choose any of the systems below. By default, this notebook uses tacc.prod.

In [3]:
# In case you don't want to use tacc.prod, here is
# a list of possible Agave tenants.
!tenants-list

3dem
agave.prod
araport.org
designsafe
iplantc.org
irec
portals
sd2e
sgci
tacc.prod
vdjserver.org


### Edit this next line to change the tenant.

In [4]:
# Set the tenant.
!tenants-init -t tacc.prod

You are now configured to interact with the APIs at https://api.tacc.utexas.edu/


## Step 2: Supply your password

In [5]:
# Run the cell below to load your AGAVE_PASSWD
# (which will be your TACC password if you use
# the tacc.prod tenant, which is the default).
# By default, this will be the password stored
# in AGAVE_PASSWD.txt. If that file does not exist
# you will be prompted for your password.
#
# If, for some reason, that password is incorrect,
# you can get the system to prompt you again by
# uncommenting the next line and removing the file.
#os.remove("AGAVE_PASSWD.txt")
readpass("AGAVE_PASSWD")

Password or secret: AGAVE_PASSWD
Reading file `AGAVE_PASSWD.txt'


## Step 3: Supply your Username and Email

In [6]:
# Fill in your Agave username to the right of
# the eqauls sign. (This will be your TACC username
# if you are using the tacc.prod tenant, which is
# the default)
setvar("""
AGAVE_USERNAME=tg457049
EMAIL=sbrandt@cct.lsu.edu
""")

AGAVE_USERNAME=tg457049
EMAIL=sbrandt@cct.lsu.edu


In [7]:
# This name will be provided for you by the user that runs the
# ResourceConfigurator program.
setvar("APP_NAME=crcollaboratory-shelob.hpc.lsu.edu-tg457049-2.0")

APP_NAME=crcollaboratory-shelob.hpc.lsu.edu-tg457049-2.0


## Step 4: Supply the name of the Agave Application you will be using.
You should obtain this from someone who as set up the CMR using the
ResourceConcifguator. By default, jobs will execute on shelob.hpc.lsu.edu
and you will need to contact sbrandt@cct.lsu.edu to gain permission to
run jobs.

## Step 5: Supply your PushBullet Token
Please go to https://www.pushbullet.com/ to obtain your token.

In [8]:
# Optional:
# The value of your PushBullet token goes here.
# As with the AGAVE_PASSWD entry above, please
# delete it if you wish to use PushBullet to
# receive notifications.
#
#os.remove("PBTOK.txt")
readpass("PBTOK")

Password or secret: PBTOK
Reading file `PBTOK.txt'


# END DATA ENTRY
If you have entered the data above correctly,
you should not need to edit anything below
this line. You will, however, need to execute
the cells below.

In [9]:
# Some commands to initialize your access to the system
setvar("CLIENT_NAME=user-${AGAVE_USERNAME}")
!clients-delete -u $AGAVE_USERNAME -p $AGAVE_PASSWD $CLIENT_NAME
!clients-create -p $AGAVE_PASSWD -S -N $CLIENT_NAME -u $AGAVE_USERNAME
!auth-tokens-create -u $AGAVE_USERNAME -p $AGAVE_PASSWD

CLIENT_NAME=user-tg457049
Successfully deleted client user-tg457049
Successfully created client user-tg457049
key: 0fQA1mXlaf2ezVe1hxcbvdnUK_Ea 
secret: 0jQRqFjWRFPnYE8nLUUJn1rR9Xsa
Token for tacc.prod:tg457049 successfully refreshed and cached for 14400 seconds
63d48af6cc3db221903b5d986f761940


In [10]:
!apps-list -v ${APP_NAME} > app.txt
import json
with open("app.txt","r") as fd:
    fi = json.load(fd)
    exec_sys=fi["_links"]["executionSystem"]["href"]
    storage_sys=fi["_links"]["storageSystem"]["href"]
setvar("""
EXEC_MACHINE={}
STORAGE_MACHINE={}
""".format(exec_sys, storage_sys))

EXEC_MACHINE=https://api.tacc.utexas.edu/systems/v2/shelob-exec-tg457049
STORAGE_MACHINE=https://api.tacc.utexas.edu/systems/v2/shelob-storage-tg457049
